In [31]:
import os
import pymysql
import pandas as pd
import numpy as np
import warnings
import string
import csv
import sys
from datetime import date
warnings.filterwarnings('ignore')

In [30]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [32]:
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

In [33]:
conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    database = "shopping_billing",
    passwd="spatil123",
#     db="python_project",
    charset='utf8mb4')
print(conn)

In [34]:
mycur = conn.cursor()

In [35]:
mycur.execute("show databases") 
myresult = mycur.fetchall()
for i in myresult:
  print(i)

('abc',)
('digikull',)
('information_schema',)
('mysql',)
('org',)
('performance_schema',)
('shopping_billing',)
('sqltest',)
('sss',)
('sys',)
('xyz',)


In [36]:
mycur.execute("use shopping_billing") 

0

In [108]:
mycur.execute("CREATE TABLE login (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(20), password varchar(10))") #Login table
mycur.execute("CREATE TABLE items (id INT AUTO_INCREMENT PRIMARY KEY, item_name CHAR(20), price float(5.3))") #items table
mycur.execute("CREATE TABLE bills (bill_id INT AUTO_INCREMENT PRIMARY KEY, c_name CHAR(20), c_phone varchar(10),bill_date date)")#bills table
mycur.execute("CREATE TABLE transcation (id INT AUTO_INCREMENT PRIMARY KEY, item_id int(5), qty int(10), bill_id int(10))")#transcation table

0

In [109]:
mycur.execute(" INSERT INTO login (id, name, password) VALUES (1, 'Ram', 12345)")
mycur.execute(" INSERT INTO items (id, item_name, price) VALUES (1, 'soap', 25.00),(2, 'chips', 10.00),(3, 'shampoo',150.00),(4, 'facewash', 79.00),(5, 'pen', 15.00),(6, 'bucket',199.00),(7, 'chair', 750.00),(8, 'pan', 250.00)")
conn.commit()#for load table data into sqlserver

In [37]:
mycur.execute("SHOW TABLES")
my=mycur.fetchall()
for i in (my):
    print(i)

('bills',)
('items',)
('login',)
('transcation',)


In [38]:
def clear():
    for _ in range(2):
        print()

In [58]:
def login():
    while True:
        clear()
        uname =input('Enter your id:')
        upass =input('Enter your pass:')
        mycur.execute('select * from login where id = "{}" and password ="{}"'.format(uname,upass))
        mycur.fetchall()
        rows = mycur.rowcount
        if rows!=1:
            print('invalid login details..try again')
            break
        else:
            print('\n   *** Login Successful ***   ')
            print('\n\npress any key to continue')
            wait = input()
        
        #if upass == password and uname ==id:
           # print('\n   *** Login Successful ***   ') 
        #else:
            #print('invalid login details..try again')
            #break

In [51]:
def last_bill_no():
    mycur.execute('select max(bill_id) from bills')
    record = mycur.fetchone()
    return record

In [52]:
def find_item(no):
    mycur.execute('select * from items where id ={}'.format(no))
    record = mycur.fetchone()
    return record

In [53]:
def item_list():
    clear()
    sql="select * from items"
    mycur.execute(sql)
    records = mycur.fetchall()
    for row in records:
        print(row)
    print('\nPress any key to continue.....')
    wait = input()

In [ ]:
def billing():
    clear()
    items =[]
    bill_no = last_bill_no()
    if bill_no[0]==None:
        bill_no =1
    else:
        bill_no = bill_no[0]+1 

    name = input('Enter customer Name :')
    phone = input('Enter Phone No :')
    today = date.today()
    while True:
        no = int(input('Enter item No (0 to stop) :'))
        if no <=0:
            break
        else:
            item = find_item(no)
            if item==None:
                print('Item Not found  ')
            else:
                qty = int(input('Enter Item Qty :'))
                item = list(item)
                item.append(qty)
                items.append(item)

    clear()
    print("                          ***  Supermart  ***                         ")
    print(" ")
    print("--------------------------------Invoice-------------------------------")
    print(' Phone: 022-2221542,                       Email: supermart@gmail.com ')
    print(' Bill No :{}                               Date :{}'.format(bill_no,today))
    print('-'*70)
    print('Customer Name :{}                          Phone No :{}'.format(name,phone))
    print('-'*70)
    print('Item Id     Item Name                Price         Qty         Amount ')
    print('-'*70)
    total =0
    for item in items:
        print('{:<10d} {:25s} {:.2f} {:>10d}          {:>.2f}'.format(item[0],item[1],item[2],item[3],item[2]*item[3]))
        total = total +item[2]*item[3]
    print('-'*70)
    print('Total Payable amount : {}'.format(total))
    print('\nPress any key to continue........')
    #insert data into tables
    sql ='insert into bills(c_name,c_phone,bill_date) values("{}","{}","{}");'.format(name,phone,today)
    mycur.execute(sql)    
    for item in items:
        sql='insert into transcation(item_id,qty,bill_id) values({},{},{});'.format(item[0],item[3],bill_no)
        mycur.execute(sql)
        conn.commit()
    wait= input()
login()
billing()



Enter your id:1
Enter your pass:123
invalid login details..try again


